In [ ]:
import pandas as pd
import re
from konlpy.tag import Mecab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from tqdm import tqdm  # 반복문 진행상황 보기
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
import numpy as np
from gensim.models.callbacks import CallbackAny2Vec
import time

In [ ]:
import pymysql
import pandas as pd
host_name = 'localhost'
host_port = 3306
username='root'
password='12345678'
database_name='news_team_4'
db = pymysql.connect(
    host=host_name,
    port=host_port,
    user=username,
    password=password,
    db=database_name,
    charset='utf8'
)
SQL='SELECT Content, Title FROM tb_news_team_4'
df=pd.read_sql(SQL, db)

In [ ]:
tagged_corpus_list = []
for i in range(len(df)):
    content = df['Content'].iloc[i]
    tag = df['Title'].iloc[i]
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=content))
    if i ==70000:
        break

In [10]:
len(tagged_corpus_list)

70001

In [9]:

class MonitorCallback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.last = time.time()

    def on_epoch_end(self, model):
        print('epoch ends {}: {}'.format(self.epoch, time.time() - self.last))
        self.epoch += 1
        self.last = time.time()


# to see all data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 450)
pd.set_option('display.colheader_justify', 'left')

# tagged_corpus_list = df['Content'].to_list()
# print(tagged_corpus_list)


In [11]:
model = Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.01, workers=6, window=2, callbacks=[MonitorCallback()])
model.build_vocab(tagged_corpus_list)
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)


model.save('trainedMDL.')
loaded_model = Doc2Vec.load('trainedMDL')


In [12]:


def cos_simimlarity(first_docu, second_docu):
    return dot(first_docu, second_docu) / (
            norm(first_docu) * norm(second_docu))



mdoel_size = len(loaded_model.dv)


similarity_scores = []
similarity_scores_list = []
for idx, content_first in enumerate(tqdm(loaded_model.dv)):
    for idx_second in range(0, mdoel_size - 1):
        try:
            # similarity_scores.append(cos_simimlarity(content_first, loaded_model.dv[idx_second]))
            cosine_result = cosine_similarity([loaded_model.dv[idx].T], [loaded_model.dv[idx_second]])
            similarity_scores.append(*cosine_result)
            # print("cosine_result : " , idx, idx_second, *cosine_result)
        except Exception as e:
            print("Exception : ", idx, ":", idx_second, ":", e)
    similarity_scores_list.append(similarity_scores)
    # loaded_model.dv['cos_similarity']

    # loaded_model.dv.vectors['similarity_scores'].iloc[idx] = similarity_scores



  0%|          | 8/53738 [00:47<89:08:56,  5.97s/it]


KeyboardInterrupt: 

In [ ]:
from turtle import dot
from gensim.models.doc2vec import Doc2Vec
from scipy.stats import norm
from tqdm import tqdm  # 반복문 진행상황 보기
loaded_model = Doc2Vec.load('trainedMDL')

def cos_simimlarity(first_docu, second_docu):
    return dot(first_docu, second_docu) / (
            norm(first_docu) * norm(second_docu))


In [ ]:
mdoel_size = len(loaded_model.dv)
similarity_scores = []
similarity_scores_list = []
for idx, content_first in enumerate(tqdm(loaded_model.dv)):
    for idx_second in range(0, mdoel_size - 1):
        try:
            # similarity_scores.append(cos_simimlarity(content_first, loaded_model.dv[idx_second]))
            cosine_result = cos_simimlarity([loaded_model.dv[idx].T], [loaded_model.dv[idx_second]])
            similarity_scores.append(*cosine_result)
            # print("cosine_result : " , idx, idx_second, *cosine_result)
        except Exception as e:
            print("Exception : ", idx, ":", idx_second, ":", e)
    similarity_scores_list.append(similarity_scores)